In [2]:
demand = []
cdemand = [] # TODO 这个cdemand是啥
fopen = open('/Users/zihanding/Developer/Psquare/newevaluation_dis/historydemand/slot20/groundtruth/24', 'r')
for line in fopen:
    line = line.strip().split(',')
    csum = 0
    one = []
    for k in line:
        one.append((float(k)))
        csum += (float(k))
    demand.append(csum) # 37 每个line所有的数据加起来是csum 从i出去的demandd
    cdemand.append(one) # 37x37 i到j的demand
fopen.close()

In [7]:
def createa(a,b,c):
    a += 1
    b += 1
    c += 12
    return a,b,c

a,b,c = 1,1,1
a,b,c = createa(a,b,c)
print(a,b,c)

2 2 13
